# Plateforme d'Acquisition et de Traitement Automatisé des Textes
### Carnet d'expérimentation
---

## Sujet : Evaluations prédicteurs sur dataset ObsInfo2307
### Comparaison labelisation humaine et chatgpt

---

# Imports

In [1]:
cd ../..

/Volumes/Geek/Work/Patat


In [2]:
import importlib
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
tqdm.pandas()

# Chargement Données

### Annotations Manuelles

In [3]:
import patat.util.gsheet

gs = patat.util.gsheet.GoogleSheet()

sheet_id = '1QSXfuWNr9Wpe7aAAluKGVRuI1CcN8erpmo0R90UTLT0'
range_name = 'Data'

df_obsinfo = gs.read_df(sheet_id,range_name)

In [4]:
labels = [
    'infox',
    'lieux, dates et personnes',
    'faits',
    'opinions',
    'subjectif',
    'propos rapporté',
    'sources citées',
    'fausse nouvelle',
    'insinuation',
    'exagération',
    'titre décalé',
]

In [5]:
import numpy as np

In [6]:
def normalize(x):
    if x == '0':
        return 0.0
    elif x== '1':
        return 1.0
    else:
        return np.NaN

In [7]:
for label in labels:
    df_obsinfo[label]=df_obsinfo[label].apply(normalize)

In [8]:
df_obsinfo['url_h'] = df_obsinfo['url_labelizer'].apply(lambda s: s[-40:])

In [9]:
df_obsinfo

,url,url_labelizer,title,infox,"lieux, dates et personnes",faits,opinions,subjectif,propos rapporté,sources citées,fausse nouvelle,insinuation,exagération,titre décalé,commentaires,observateur,rater,url_h
0,https://lesakerfrancophone.fr/la-relation-entr...,https://app.observatoire.info/labelize?url_h=c...,La relation entre la technologie et la religion,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,vraiment indigeste pour commencer !,FB,rater1,c13175936764a345ec21bc398fe750a41496078e
1,https://www.breizh-info.com/2021/01/27/157958/...,https://app.observatoire.info/labelize?url_h=0...,"Confinement. Les habitants de Brest, Morlaix e...",0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,,FB,rater1,0552561b3c46f64febfbc3a77c301f085a98b966
2,https://reseauinternational.net/la-chine-le-pr...,https://app.observatoire.info/labelize?url_h=3...,La Chine : Le premier marché mondial de Smartp...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,,FB,rater1,361a904687dd4b1abb189717de7341ce162d21c7
3,https://lezarceleurs.blogspot.com/2021/12/emma...,https://app.observatoire.info/labelize?url_h=4...,"Emmanuel à Olivier : « Tiens bon, on les aura ...",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,Article dupliqué (je l'avais manqué),FB,rater1,433b403d386d5eba5dff7d45044cdb0fe7862972
4,https://lesakerfrancophone.fr/selon-ubs-les-pr...,https://app.observatoire.info/labelize?url_h=3...,"Selon UBS, les « propriétés d’assurance tant d...",0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,,FB,rater1,3494ad0901a73fc6ceb2f74ac4ac1930af42b363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,https://lemediaen442.fr/loi-climat-le-gouverne...,https://app.observatoire.info/labelize?url_h=d...,Loi climat : le gouvernement va exempter Amazo...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,,MC,rater8,d04a526d60fa6d67f61c7abcaf45092b8217a24d
796,https://www.breizh-info.com/2017/11/10/81481/q...,https://app.observatoire.info/labelize?url_h=1...,Le QI peut-il être augmenté par l’éducation ? ...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,,MC,rater8,14ed4715949dc7767359674df1a076bb1e0e417b
797,https://lesmoutonsrebelles.com/secte-sexuelle-...,https://app.observatoire.info/labelize?url_h=2...,Secte sexuelle : l’ex-star de Smallville Aliso...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,,MC,rater8,2841a1db742a25419c20d4a56eb65d8c05d92f65
798,https://lesalonbeige.fr/sacre-de-charles-iii-p...,https://app.observatoire.info/labelize?url_h=7...,Sacre de Charles III par SAR le Prince Charles...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,,MC,rater8,7f8a6372ed4084fc88263c03e3b9186f082e2162


## Moyennes et accords

In [10]:
df_mean = df_obsinfo.pivot_table(index='url_h',values=labels,aggfunc='mean')

In [11]:
df_accord = df_mean.apply(lambda x : abs(2*x-1))

## Chargement prédicteurs

In [12]:
import pickle

with open('data/tmp/231023-PredictorsBow.pkl','rb') as file:
    predictors = pickle.load(file)

In [13]:
predictors.keys()

dict_keys(['infox', 'entites_nommees', 'ouverture_esprit', 'faits', 'opinions', 'propos_raportes', 'sources_citees', 'fausse_nouvelle', 'insinuations', 'exageration'])

In [14]:
labels = ['infox', 'entites_nommees', 'faits', 'opinions', 'propos_raportes', 'sources_citees',
          'fausse_nouvelle', 'insinuations', 'exageration']

In [15]:
df_obsinfo.keys()

Index(['url', 'url_labelizer', 'title', 'infox', 'lieux, dates et personnes',
       'faits', 'opinions', 'subjectif', 'propos rapporté', 'sources citées',
       'fausse nouvelle', 'insinuation', 'exagération', 'titre décalé',
       'commentaires', 'observateur', 'rater', 'url_h'],
      dtype='object')

In [19]:
label_translator={
    'lieux, dates et personnes':'entites_nommees',
    'propos rapporté':'propos_raportes',
    'sources citées':'sources_citees',
    'exagération':'exageration', 
    'insinuation':'insinuations',
    'fausse nouvelle':'fausse_nouvelle',
    
}

In [20]:
df_obsinfo = df_obsinfo.rename(columns=label_translator)

In [21]:
df_obsinfo[labels]

,infox,entites_nommees,faits,opinions,propos_raportes,sources_citees,fausse_nouvelle,insinuations,exageration
0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
795,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
796,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
797,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
798,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [22]:
for label in labels:
    print(predictors[label])

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=100, max_iter=1000, multi_class='ovr',
                                    random_state=42))])
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=100, max_iter=1000, multi_class='ovr',
                                    random_state=42))])
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=100, max_iter=1000, multi_class='ovr',
                                    random_state=42))])
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=100, max_iter=1000, multi_class='ovr',
                                    random_state=42))])
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregres

## Performances  - Moyenne
- On ne considère que les labels pour lesquels accord >= seuil_accord.

In [ ]:
#seuil_accord = 1 # 8/8 observateurs
#seuil_accord = 0.75 # 7/8 observateurs
#seuil_accord = 0.50 # 6/8 observateurs
#seuil_accord = 0.25 # 5/8 observateurs
seuil_accord = 0 # toutes observations prises en compte

In [ ]:
pred_score = []
mean_pred = []
for index,row in df_gpt.iterrows():
    url_h=row['url_h']
    score_item = {'url_h':url_h}
    pred_item = {'url_h':url_h}
    for label in labels:
        if df_accord [label][index] >= seuil_accord:
            score_item[label]=row[label]-df_mean[label][url_h]
            pred_item[label] = round(df_mean[label][url_h])
    gpt_score.append(score_item)
    mean_pred.append(pred_item)
        

In [ ]:
df_gpt_score = pd.DataFrame(gpt_score)
df_mean_pred = pd.DataFrame(mean_pred)

In [ ]:
from sklearn import metrics

In [ ]:
results = {}
for label in labels:
    indexes = df_mean_pred[label].notna()
    y_test = df_mean_pred[label][indexes]
    y_pred = df_gpt[label][indexes]  
    results[label] = {}
    results[label]['accuracy']=metrics.accuracy_score(y_test, y_pred)
    results[label]['precision']=metrics.precision_score(y_test, y_pred)
    results[label]['f1']=metrics.f1_score(y_test, y_pred)
    results[label]['recall']=metrics.recall_score(y_test, y_pred)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred,normalize='all')
    results[label]['false_positive']=cnf_matrix[0,1]
    results[label]['false_negative']=cnf_matrix[1,0]
    results[label]['count']=len(y_test)


In [ ]:
pd.set_option('display.precision',3)

In [ ]:
df_results = pd.DataFrame(results).T
df_results

In [ ]:
plot = sns.scatterplot(df_results[['accuracy','f1']])
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)

In [ ]:
plot = sns.scatterplot(df_results[['precision','recall']])
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)

In [ ]:
plot = sns.scatterplot(df_results[['false_positive','false_negative']])
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)